In [29]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.datasets import CocoDetection
from torch.utils.data import DataLoader
from torchvision import transforms

from engine import train_one_epoch
import utils

from func import collate_fn


In [30]:
#データセットの読み込み
# Define the transformation
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor()
])


# Load the COCO formatted dataset
coco_dataset = CocoDetection(root='../project-1-at-2023-07-21-05-16-d6bd696f', annFile='../project-1-at-2023-07-21-05-16-d6bd696f/result.json', transform=transform)

# Create a DataLoader
data_loader = torch.utils.data.DataLoader(
    coco_dataset, 
    batch_size=2, 
    shuffle=True, 
    num_workers=4,
    collate_fn=collate_fn)


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [31]:
# Load a pre-trained model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Replace the classifier with a new one, that has num_classes which is user-defined
num_classes = 2  # 1 class (your object) + background
in_features = model.roi_heads.box_predictor.cls_score.in_features

# Replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


/Users/s.takabayashi/OCR-Sample/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/s.takabayashi/OCR-Sample/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [32]:
# Move model to the right device
# Define the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Move model to the right device
model.to(device)

# Construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# And a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    # Train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # Update the learning rate
    lr_scheduler.step()

AttributeError: 'list' object has no attribute 'items'